# FastAval Model Training

This notebook trains a BERT-based model with a linear layer to evaluate LLM responses and generate a risk score from 0 to 1.

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

## Define Model Architecture

In [ ]:
class FastAvalModel(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased"):
        super(FastAvalModel, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.score_layer = nn.Linear(self.bert.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        score = self.score_layer(pooled_output)
        return self.sigmoid(score).squeeze()

## Dataset Preparation

In [ ]:
class EvaluationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Combine system prompt, user prompt and response
        text = f"System: {row['system_prompt']}\n\nUser: {row['user_prompt']}\n\nResponse: {row['response']}"
        
        # Tokenize
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'score': torch.tensor(row['score'], dtype=torch.float)
        }

## Load Data from Parquet

In [ ]:
# Set the path to your parquet files
data_path = "/home/eduardo/Desktop/Others/Adapta/vizeval/synthetic_data/"

# Load data from parquet files
df = pd.read_parquet(data_path)

# Display sample data
df.head()

In [ ]:
# Check data structure
print(f"Dataset shape: {df.shape}")
print("\nColumns:")
for col in df.columns:
    print(f"- {col}")

# Basic statistics for the score column
print("\nScore statistics:")
print(df['score'].describe())

## Prepare Training and Validation Sets

In [ ]:
# Split data into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)

print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Create datasets
train_dataset = EvaluationDataset(train_df, tokenizer)
val_dataset = EvaluationDataset(val_df, tokenizer)

# Create data loaders
batch_size = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

## Training Function

In [ ]:
def train_model(model, train_loader, val_loader, epochs=3, learning_rate=2e-5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    model.to(device)
    
    # Define optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )
    
    # Define loss function
    criterion = nn.MSELoss()
    
    # Training loop
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        
        # Training phase
        model.train()
        train_loss = 0
        
        progress_bar = tqdm(train_loader, desc="Training")
        for batch in progress_bar:
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            scores = batch['score'].to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            
            # Calculate loss
            loss = criterion(outputs, scores)
            train_loss += loss.item()
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            
            # Update progress bar
            progress_bar.set_postfix({"loss": loss.item()})
        
        avg_train_loss = train_loss / len(train_loader)
        print(f"Average training loss: {avg_train_loss:.4f}")
        
        # Validation phase
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            progress_bar = tqdm(val_loader, desc="Validation")
            for batch in progress_bar:
                # Move batch to device
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                scores = batch['score'].to(device)
                
                # Forward pass
                outputs = model(input_ids, attention_mask)
                
                # Calculate loss
                loss = criterion(outputs, scores)
                val_loss += loss.item()
                
                # Update progress bar
                progress_bar.set_postfix({"loss": loss.item()})
        
        avg_val_loss = val_loss / len(val_loader)
        print(f"Average validation loss: {avg_val_loss:.4f}")
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), "best_fastval_model.pt")
            print("Saved best model!")
    
    return model

## Train the Model

In [ ]:
# Initialize model
model = FastAvalModel()

# Train model
trained_model = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=1,
    learning_rate=2e-5
)

## Export to TorchScript

In [ ]:
def export_to_torchscript(model, save_path="fastval_model.pt"):
    # Set model to evaluation mode
    model.eval()
    
    # Create example inputs for tracing
    example_input_ids = torch.randint(0, 30522, (1, 512))  # BERT vocab size is 30522
    example_attention_mask = torch.ones(1, 512)
    
    # Trace the model
    traced_model = torch.jit.trace(model, (example_input_ids, example_attention_mask))
    
    # Save the traced model
    torch.jit.save(traced_model, save_path)
    print(f"Model exported to TorchScript format and saved at {save_path}")
    
    return save_path

In [ ]:
# Export the trained model to TorchScript format
model_path = export_to_torchscript(trained_model, "fastval_model.pt")